In [ ]:
# coding: utf-8
import tensorflow as tf
import numpy as np
import os
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('ckpt_path', '/home/artemis/下载/车牌号训练/homework/CNN',"""存放模型的目录""")

tf.app.flags.DEFINE_string('model_name', 'num_letter',"""模型的名称""")
  


def read_and_decode(filename, batch_size=300):
    # 生成一个queue队列
    filename_queue = tf.train.string_input_producer([filename])

    reader = tf.TFRecordReader()
    # 返回文件名和文件
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label': tf.FixedLenFeature([], tf.int64),
                                           'img_raw': tf.FixedLenFeature([], tf.string),
                                       })  # 将image数据和label取出来

    img = tf.decode_raw(features['img_raw'], tf.uint8)
    img = tf.reshape(img, [1152])
    img = tf.cast(img, tf.float32) * (1. / 255) - 0.5  # 在流中抛出img张量
    label = tf.cast(features['label'], tf.int32)  # 在流中抛出label张量

    image_batch, label_batch = tf.train.shuffle_batch([img, label],
                                                      batch_size=batch_size,
                                                      capacity=1000,
                                                      min_after_dequeue=50)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        coord = tf.train.Coordinator()
        thread = tf.train.start_queue_runners(coord=coord)
        image_ = sess.run(image_batch)
        label_ = sess.run(label_batch)

        coord.request_stop()
        coord.join(thread)
        new_label = np.zeros([batch_size, 34], dtype=int)
        for i in range(batch_size):
            new_label[i][label_[i]] = 1
    return image_, new_label


# 读取数据
train_data = read_and_decode('train.tfrecords', 500)
print(train_data[0][0])
print(train_data[1][0])


def weight_variable(shape):  
    initial = tf.truncated_normal(shape, stddev=0.1)  
    return tf.Variable(initial)  
  

def bias_variable(shape):  
    initial = tf.constant(0.1, shape=shape)  
    return tf.Variable(initial)  
  

def conv2d(x, W):  
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')  
  

def max_pool_2x2(x):  
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')  
   
ckpt = tf.train.latest_checkpoint(FLAGS.ckpt_path)
if not ckpt:
    if not os.path.exists(FLAGS.ckpt_path):
        os.mkdir(FLAGS.ckpt_path)

  
sess = tf.InteractiveSession()  
  
x = tf.placeholder("float", shape=[None, 1152])
y_ = tf.placeholder("float", shape=[None, 34])

x_image = tf.reshape(x, [-1, 48, 24, 1])
keep_prob = tf.placeholder(tf.float32)


# 第一层卷积池化
W_conv1 = weight_variable([3, 3, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  
h_pool1 = max_pool_2x2(h_conv1)  

# 第二层卷积池化
W_conv2 = weight_variable([3, 3, 32, 64])
b_conv2 = bias_variable([64])  
  
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  
h_pool2 = max_pool_2x2(h_conv2)  
# h_pool2_flat = tf.reshape(h_pool2, [-1, 12*6*64])

# 第三层卷积池化
W_conv3 = weight_variable([3, 3, 64, 128])
b_conv3 = bias_variable([128])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)
h_pool3_flat = tf.reshape(h_pool3, [-1, 6 * 3 * 128])

# 全连接层
# Now image size is reduced to 6*3
W_fc1 = weight_variable([6 * 3 * 128, 625])
b_fc1 = bias_variable([625])

h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)  

W_fc2 = weight_variable([625, 34])
b_fc2 = bias_variable([34])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))

train_step = tf.train.AdamOptimizer().minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))   # tf.argmax(y_, 1)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver=tf.train.Saver(max_to_keep=2)
sess.run(tf.global_variables_initializer())

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(FLAGS.ckpt_path,sess.graph)
    
    ckpt = tf.train.latest_checkpoint(FLAGS.ckpt_path)
    step=0
    if ckpt:
        saver.restore(sess=sess,save_path=ckpt)
        step = int(ckpt[len(os.path.join(FLAGS.ckpt_path, FLAGS.model_name)) + 1:])


    for i in range(200):    
        if i % 1 == 0:
            train_accuracy = sess.run(
                accuracy,
                feed_dict={x: train_data[0], y_: train_data[1], keep_prob: 1.0})
            print("step %d, training accuracy %.3f" % (i, train_accuracy), end=' ')
            print('cost : ', sess.run(
                cross_entropy,
                feed_dict={x: train_data[0], y_: train_data[1], keep_prob: 1.0}))
            ckptname=os.path.join(FLAGS.ckpt_path, FLAGS.model_name)
            saver.save(sess,ckptname,global_step=i)
        train_step.run(feed_dict={x: train_data[0], y_: train_data[1], keep_prob: 0.5})
    
    print("Training finished")  
